In [ ]:
import pandas as pd
# pd.options.display.max_rows = 20
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from IPython.display import clear_output
from itertools import islice
HOME = "/content/drive/MyDrive/Colab_Notebooks/recommendation"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
datasets_location = HOME +"/datasets/"
chunk_size = 20000  # Adjust the chunk size as needed
raw_chunks = pd.read_csv(datasets_location + 'animelist.csv', chunksize=chunk_size)
chunk_count = number_of_chunks = sum([1 for _ in islice(raw_chunks, None)])
print(chunk_count)
raw_chunks = pd.read_csv(datasets_location + 'animelist.csv', chunksize=chunk_size)

5462


In [ ]:
animeDF = pd.read_csv(datasets_location + 'anime.csv')
columns = ["Genres"]
animeDFObj = {}
for chunkColumn in columns:
    animeDFObj["length"] = 0
    animeDFObj["columns"] = {}
    animeDFObj["columnsList"] = ['action', 'adventure', 'comedy', 'drama', 'sci-fi', 'space', 'mystery', 'shounen', 'police', 'supernatural', 'magic', 'fantasy', 'sports', 'josei', 'romance', 'slice of life', 'cars', 'seinen', 'horror', 'psychological', 'thriller', 'super power', 'martial arts', 'school', 'ecchi', 'vampire', 'military', 'historical', 'dementia', 'mecha', 'demons', 'samurai', 'game', 'shoujo', 'harem', 'music', 'shoujo ai', 'shounen ai', 'kids', 'hentai', 'parody', 'yuri', 'yaoi']

    def Genres(row, animeDFObj, isAdd):
        text = row[chunkColumn]
        # print(text)
        text = text.split(",")
        if isAdd:
            animeDFObj["length"] += len(text)
        if isAdd:
            for column in text:
                column = column.lower().strip()
                if column == "unknown":
                    continue
                # print(animeDFObj["columns"][column])
                if column in animeDFObj["columns"].keys():
                    animeDFObj["columns"][column] +=  1
                else:
                    animeDFObj["columns"][column] = 1
        else:
            for column in text:
                column = column.lower().strip()
                if column == "unknown":
                    continue
                # row[column] = round(animeDFObj["columns"][column] / animeDFObj["length"],3)
                row[column] = 1
        return row

    animeDF.apply(lambda x: Genres(x, animeDFObj, True),axis = 1)

    animeDFObj["columnsList"] = animeDFObj["columns"].keys()

    for column in animeDFObj["columnsList"]:
        if column == "unknown":
            continue
        animeDF[column] = np.zeros((animeDF.shape[0], 1), dtype=int)


    animeDF = animeDF.apply(lambda x: Genres(x,animeDFObj,False),axis = 1)

In [ ]:
pd.read_csv(datasets_location + 'watching_status.csv')

,status,description
0,1,Currently Watching
1,2,Completed
2,3,On Hold
3,4,Dropped
4,6,Plan to Watch


In [ ]:
# watching_status = pd.read_csv(datasets_location + 'watching_status.csv')
# watching_statusObj = {}
# for i in watching_status[["status"," description"]].values:
#     # print(i)
#     watching_statusObj[i[0]] = i[1]
# watching_statusObj[5] = "unknown"
# print(watching_statusObj)
watching_statusObj = {1: 1, 2: 1, 3: 0, 4: 0, 6: 0, 5: 0}
watching_statusObj

{1: 1, 2: 1, 3: 0, 4: 0, 6: 0, 5: 0}

In [ ]:
def calculateRemaining(primary,isInc):
    remaining = 1 - primary if primary >= 0.5 else primary

    if round(primary,3) == 1:
        remaining = 0.5 if isInc == 0 else 0

    if round(primary,3) == 0:
        remaining = 0 if isInc == 0 else 0.5

    if (isInc == 0.001 or isInc == 0):
        result = primary - (remaining / 2)
    else:
        result = primary + (remaining / 2)
    result = 0 if result < 0 else result;
    # print(primary,isInc,"  ",remaining,result)
    return round(result, 4)

In [ ]:
temp_count = 0

def preprocess_chunk(chunk):
    chunk = pd.concat([chunk, pd.DataFrame(0, index=chunk.index, columns=animeDFObj["columnsList"])], axis=1)
    chunk["watching_status"].fillna(5, inplace=True)
    chunk["watching_status"] = chunk["watching_status"].apply(lambda x: watching_statusObj.get(x, 5))
    return chunk

def update_chunk_with_selected_data(user, chunk, columnsList):
    temp = None
    for index, row in user.iterrows():
        anime_id = row["anime_id"]
        selected_data = animeDF.loc[animeDF["MAL_ID"] == anime_id, columnsList]
        if temp is not None:
            selected_data = selected_data.apply(lambda x: [calculateRemaining(temp[x.name].values[0],x.values[0])],axis = 0)
        temp = selected_data
        chunk.loc[(chunk["user_id"] == row["user_id"]) & (chunk["anime_id"] == anime_id), columnsList] = selected_data.values
    return chunk

# Assuming 'raw_chunks' is a list of data chunks
optimized_chunks = []

for raw_chunk in raw_chunks:
    # if (temp_count < 2):
    #   continue
    processed_chunk = preprocess_chunk(raw_chunk)
    grouped_users = processed_chunk.groupby("user_id")
    for _, user_group in grouped_users:
        user_columns = animeDFObj["columnsList"]
        user_data = animeDF.loc[animeDF["MAL_ID"] == user_group["anime_id"].iloc[0], user_columns]
        processed_chunk = update_chunk_with_selected_data(user_group, processed_chunk, user_columns)
    optimized_chunks.append(processed_chunk)
    temp_count += 1
    print(temp_count/chunk_count)
    if (temp_count >= 5):
      break

0.0001830831197363603
0.0003661662394727206
0.0005492493592090809
0.0007323324789454412
0.0009154155986818016


In [ ]:
output_file = "optimized_chunks.csv"

# Combine all optimized chunks into a single DataFrame
combined_optimized_chunks = pd.concat(optimized_chunks, ignore_index=True)

result = pd.merge(combined_optimized_chunks, animeDF,left_on='anime_id', right_on='MAL_ID', how='inner')
# Save the combined DataFrame to a CSV file
result.to_csv(datasets_location + output_file, index=False)

print(f"Optimized chunks saved to {output_file}")

Optimized chunks saved to optimized_chunks.csv


In [ ]:
result.shape

(100000, 126)